In [4]:
from chain import UniversalChain

chain = UniversalChain("gpt-4o-mini", use_history=True)
questions = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/ Can you read the content?",
]
for i, question in enumerate(questions, 1):
    print(f"Question {i}:\n{question}")
    response = chain.generate_response(question)
    print()
    print(f"Response {i}:\n{response}")
    # chain.display_response(response)
    if i < len(questions):
        print("---")

Question 1:
https://lilianweng.github.io/posts/2023-06-23-agent/ Can you read the content?

Response 1:
I'm unable to access external websites directly, including the one you provided. However, if you can share specific content or information from that page, I would be happy to help you with it!
